In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import pickle
import gc
import os
import time
import multiprocessing as mp
import logging
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm, tqdm_notebook, _tqdm_notebook, tqdm_pandas
tic = time.time()
SAVE_PATH = '/home/huangqiao/DataMiningCode/project-expert/data/feats'
if not os.path.exists(SAVE_PATH):
    print('create dir: %s' % SAVE_PATH)
    os.mkdir(SAVE_PATH)

In [2]:
#####################################################################################################################################
######################################################user###########################################################################

In [99]:
with open('pkl/user.pkl','rb') as file:
    user=pickle.load(file)

In [100]:
logging.info("user %s", user.shape)

In [101]:
def parse_str(d):
    return np.array(list(map(float, d.split())))

In [102]:
with open('pkl/topic.pkl', 'rb') as file:
    topicmap = pickle.load(file)
topicmap.shape

(100000, 2)

In [103]:
topic_vector_dict = dict(zip(np.array(topicmap['id']), np.array(topicmap['embed'])))

type(topic_vector_dict.keys())

dict_keys

In [104]:
#求话题向量平均值
def topic2v(x):
    try:
        tmp = topic_vector_dict[x[0]]
    except:
        tmp = np.zeros(64)
    for i in x[1:]:
        tmp = tmp + topic_vector_dict[i]
    if len(tmp) == 0:
        return np.zeros(64)
    return (tmp / len(x))

In [105]:
user.head()

,uid,gender,freq,uf_b1,uf_b2,uf_b3,uf_b4,uf_b5,uf_c1,uf_c2,uf_c3,uf_c4,uf_c5,score,follow_topic,inter_topic
0,M1934753188,1,1,0,1,0,1,0,1190,130,396,1438,1,764,[540],"{21107: 1.7915097, 405: 1.6123838, 4436: 1.518..."
1,M595924114,1,0,0,0,0,1,1,486,170,0,1311,1,671,"[44126, 15940, 839, 8978, 2934, 1113, 3914, 12...","{18016: 2.0650618, 2384: 1.2503042, 1142: 1.13..."
2,M1473482940,0,4,0,1,0,1,0,294,190,73,758,1,454,"[30874, 2113, 8656, 21, 523, 8, 116, 5727, 68,...","{46: 1.330939, 2159: 1.1296458, 379: 1.1241927..."
3,M578477092,1,0,1,1,0,1,0,2282,232,237,864,1,588,"[946, 7323, 297, 2660, 36067, 53107, 2654, 507...","{15918: 1.9479566, 8106: 1.8578106, 4787: 1.58..."
4,M1088794709,1,4,0,1,0,0,0,2113,190,32,758,1,361,"[582, 558, 28776, 5186, 9081, 2419, 2693, 2299...","{65: 1.5992582, 867: 1.3179373, 93: 1.2095009,..."


In [106]:
tqdm.pandas(desc="topic2v...")
user['follow_topic_vector']=user['follow_topic'].progress_apply(lambda x:topic2v(x))
print('finished!')

topic2v...: 100%|██████████| 1931654/1931654 [00:43<00:00, 44154.63it/s]


finished!


In [107]:
def topic_interest2v(x):
    if len(x)==0:
        return np.zeros(64)
    else:
        tmp=np.zeros(64)
        for i in x:
            tmp = tmp + topic_vector_dict[i]*x[i]
        return (tmp / len(x))

tqdm.pandas(desc="topic_interest2v...")
user['inter_topic_vector']=user['inter_topic'].progress_apply(lambda x:topic_interest2v(x))
print('finished!')

topic_interest2v...: 100%|██████████| 1931654/1931654 [00:38<00:00, 49557.91it/s]

finished!


In [108]:
user.head()

,uid,gender,freq,uf_b1,uf_b2,uf_b3,uf_b4,uf_b5,uf_c1,uf_c2,uf_c3,uf_c4,uf_c5,score,follow_topic,inter_topic,follow_topic_vector,inter_topic_vector
0,M1934753188,1,1,0,1,0,1,0,1190,130,396,1438,1,764,[540],"{21107: 1.7915097, 405: 1.6123838, 4436: 1.518...","[6.125305, -1.4180568, -1.3143845, -2.9268239,...","[0.5255145018431924, 2.8507211193389996, 0.571..."
1,M595924114,1,0,0,0,0,1,1,486,170,0,1311,1,671,"[44126, 15940, 839, 8978, 2934, 1113, 3914, 12...","{18016: 2.0650618, 2384: 1.2503042, 1142: 1.13...","[0.30575216599999994, -0.9980522930666669, 0.0...","[-1.9947685124736019, 1.3387408371544118, -0.3..."
2,M1473482940,0,4,0,1,0,1,0,294,190,73,758,1,454,"[30874, 2113, 8656, 21, 523, 8, 116, 5727, 68,...","{46: 1.330939, 2159: 1.1296458, 379: 1.1241927...","[0.1508028030833333, 0.7434654864583335, -0.25...","[-0.25824397617484, 0.6167275103940157, -0.075..."
3,M578477092,1,0,1,1,0,1,0,2282,232,237,864,1,588,"[946, 7323, 297, 2660, 36067, 53107, 2654, 507...","{15918: 1.9479566, 8106: 1.8578106, 4787: 1.58...","[1.4605925141999998, -0.33966116570100013, 0.3...","[1.4908855224214492, 1.4438666038597823, 1.179..."
4,M1088794709,1,4,0,1,0,0,0,2113,190,32,758,1,361,"[582, 558, 28776, 5186, 9081, 2419, 2693, 2299...","{65: 1.5992582, 867: 1.3179373, 93: 1.2095009,...","[0.9643650885294115, 0.31516375361764704, 0.82...","[1.5879302100003165, -1.1453530874701519, -0.7..."


In [109]:
user.shape

(1931654, 18)

In [110]:
def listi(x,i):
    return x[i]

In [ ]:
for i in range(64):
    col_name = 'topic_vector_{}'.format(str(i))
    tqdm.pandas(desc="topic_interest2v...")
    user[col_name]=user['follow_topic_vector'].apply(lambda x:listi(x,i))

In [ ]:
for i in range(64):
    col_name = 'topic_interestvector_{}'.format(str(i))
    tqdm.pandas(desc="topic_interest2v...")
    user[col_name]=user['inter_topic_vector'].apply(lambda x:listi(x,i))

In [ ]:
user.head()

In [ ]:
user.shape

In [ ]:
with open('pkl/user_feat.pkl','wb') as file:
    pickle.dump(user,file)

In [ ]:
#######################################################添加

In [ ]:
PATH = '/home/huangqiao/DataMiningCode/project-expert/data'

In [ ]:

user = pd.read_csv(os.path.join(PATH, 'member_info_0926.txt'), 
                          names=['uid', 'gender', 'creat_keyword', 'level', 'hot', 'reg_type', 'reg_plat', 'freq', 'uf_b1', 'uf_b2',
                'uf_b3', 'uf_b4', 'uf_b5', 'uf_c1', 'uf_c2', 'uf_c3', 'uf_c4', 'uf_c5', 'score', 'follow_topic',
                'inter_topic'], sep='\t')
user.head()

In [ ]:
def parse_list_1(d):
    if d == '-1':
        return [0]
    return list(map(lambda x: int(x[1:]), str(d).split(',')))

In [ ]:
user['creat_keyword'] = user['creat_keyword'].apply(parse_list_1)
user.head()

In [ ]:
with open('pkl/word.pkl', 'rb') as file:
    word = pickle.load(file)
word.shape

In [ ]:
word_vector_dict = dict(zip(np.array(word['id']), np.array(word['embed'])))

In [ ]:
def w2v(x):
    try:
        tmp = word_vector_dict[x[0]]
    except:
        tmp = np.zeros(64)
    for i in x[1:]:
        tmp = tmp + word_vector_dict[i]
    if len(tmp) == 0:
        return np.zeros(64)
    return (tmp / len(x))

In [ ]:
tqdm.pandas(desc="w2v...")
user['keyword_vector']=user['creat_keyword'].progress_apply(lambda x:w2v(x))

In [ ]:
user.head()

In [ ]:
def listi(x,i):
    return x[i]

In [ ]:
for i in range(64):
    col_name = 'keyword_vector_{}'.format(str(i))
    tqdm.pandas(desc="w2v...")
    user[col_name]=user['keyword_vector'].apply(lambda x:listi(x,i))

In [ ]:
user.shape

In [ ]:
columns=['uid']
for i in range(64):
    columns.append('keyword_vector_{}'.format(i))
keyword_vector=user[columns]

In [ ]:
keyword_vector.head()

In [ ]:
with open('pkl/user_keyword_feat.pkl','wb') as file:
    pickle.dump(keyword_vector,file)

In [ ]:
######################################################################################################################################
###############################################question############################################################################

In [ ]:
###################################################################加上topic的64维

In [ ]:
with open('pkl/question_info.pkl', 'rb') as file:
    ques = pickle.load(file)
ques.shape

In [ ]:
ques.head()

In [ ]:
tqdm.pandas(desc="topic2v...")
ques['topic_vector']=ques['topic'].progress_apply(lambda x:topic2v(x))
print('finished!')

In [ ]:
def listi(x,i):
    return x[i]

In [ ]:
for i in range(64):
    col_name = 'questopic_vector_{}'.format(str(i))
    tqdm.pandas(desc="topic_interest2v...")
    ques[col_name]=ques['topic_vector'].apply(lambda x:listi(x,i))

In [ ]:
ques.head()

In [ ]:
ques.shape

In [ ]:
with open('pkl/ques_feat.pkl','wb') as file:
    pickle.dump(ques,file)

In [ ]:
##############################################加上title_word切词的64维  title_t2

In [ ]:
with open('pkl/ques_feat.pkl', 'rb') as file:
    ques = pickle.load(file)

In [ ]:
ques.head()

In [ ]:
ques.shape

In [ ]:
with open('pkl/word.pkl', 'rb') as file:
    word = pickle.load(file)
word.shape

In [ ]:
word_vector_dict = dict(zip(np.array(word['id']), np.array(word['embed'])))

In [ ]:
def w2v(x):
    try:
        tmp = word_vector_dict[x[0]]
    except:
        tmp = np.zeros(64)
    for i in x[1:]:
        tmp = tmp + word_vector_dict[i]
    if len(tmp) == 0:
        return np.zeros(64)
    return (tmp / len(x))

In [ ]:
tqdm.pandas(desc="w2v...")
ques['title_w_vector']=ques['title_t2'].progress_apply(lambda x:w2v(x))

In [ ]:
ques.head()

In [ ]:
def parse_str(d):
    return np.array(list(map(float, d.split())))

In [ ]:
def listi(x,i):
    return x[i]

In [ ]:
from tqdm import tqdm, tqdm_notebook, _tqdm_notebook, tqdm_pandas
for i in range(64):
    col_name = 'title_w_vector_{}'.format(str(i))
    tqdm.pandas(desc="topic_interest2v...")
    ques[col_name]=ques['title_w_vector'].apply(lambda x:listi(x,i))

In [ ]:
ques.head()

In [ ]:
###################################################################加上内容切词  desc_t2的64维

In [ ]:
tqdm.pandas(desc="w2v...")
ques['desc_w_vector']=ques['desc_t2'].progress_apply(lambda x:w2v(x))

In [ ]:
for i in range(64):
    col_name = 'desc_w_vector_{}'.format(str(i))
    tqdm.pandas(desc="topic_interest2v...")
    ques[col_name]=ques['desc_w_vector'].apply(lambda x:listi(x,i))

In [ ]:
ques.head()

In [ ]:
with open('pkl/ques_feat.pkl','wb') as file:
    pickle.dump(ques,file)

In [ ]:
############################################################加上内容单字切词desc_t1的64维

In [ ]:
with open('pkl/ques_feat.pkl', 'rb') as file:
    ques = pickle.load(file)

In [ ]:
ques.head()

In [ ]:
with open('pkl/single_word.pkl', 'rb') as file:
    single_word = pickle.load(file)
single_word.shape

In [ ]:
single_word_vector_dict = dict(zip(np.array(single_word['id']), np.array(single_word['embed'])))

In [ ]:
def sw2v(x):
    try:
        tmp = single_word_vector_dict[x[0]]
    except:
        tmp = np.zeros(64)
    for i in x[1:]:
        tmp = tmp + single_word_vector_dict[i]
    if len(tmp) == 0:
        return np.zeros(64)
    return (tmp / len(x))

In [ ]:
tqdm.pandas(desc="sw2v...")
ques['desc_sw_vector']=ques['desc_t1'].progress_apply(lambda x:sw2v(x))

In [ ]:
ques.head()

In [ ]:
def parse_str(d):
    return np.array(list(map(float, d.split())))

In [ ]:
def listi(x,i):
    return x[i]

In [ ]:
from tqdm import tqdm, tqdm_notebook, _tqdm_notebook, tqdm_pandas
for i in range(64):
    col_name = 'desc_sw_vector_{}'.format(str(i))
    tqdm.pandas(desc="sw2v...")
    ques[col_name]=ques['desc_sw_vector'].apply(lambda x:listi(x,i))

In [ ]:
ques.shape

In [ ]:
############################################################加上内容单字切词desc_t1的64维

In [ ]:
tqdm.pandas(desc="sw2v...")
ques['title_sw_vector']=ques['title_t1'].progress_apply(lambda x:sw2v(x))

In [ ]:
for i in range(64):
    col_name = 'title_sw_vector_{}'.format(str(i))
    tqdm.pandas(desc="sw2v...")
    ques[col_name]=ques['title_sw_vector'].apply(lambda x:listi(x,i))

In [ ]:
ques.shape

In [ ]:
with open('pkl/ques_feat.pkl','wb') as file:
    pickle.dump(ques,file)

In [ ]:
######################################################################################################################################
###############################################user_question_similary############################################################################

In [ ]:
with open('pkl/ques_feat.pkl', 'rb') as file:
    ques = pickle.load(file)
ques.shape



In [ ]:
columns=['qid']

columns.append('topic_vector')
ques_topic=ques[columns]
ques_topic.head()

In [ ]:
with open('pkl/user_feat.pkl', 'rb') as file:
    user = pickle.load(file)
user.shape

In [ ]:
columns=['uid']

columns.append('inter_topic_vector')
user_topic=user[columns]
user_topic.head()

In [ ]:
uqid_sim=pd.merge(ques_topic,user_topic,on='qid')
uqid_sim.head(100)

In [ ]:
#################################################################################################################################
#################################################merge(ans,ques)的topic#####################################################################

In [ ]:
# 加载问题
with open('pkl/question_info.pkl', 'rb') as file:
    ques = pickle.load(file)
    
columns=['qid']
columns.append('topic')
ques_topic=ques[columns]
ques_topic.head()

In [ ]:
# 加载回答
with open('pkl/answer_info.pkl', 'rb') as file:
    ans = pickle.load(file)
    
columns=['qid']
columns.append('uid')
ans_topic=ans[columns]
ans_topic.head()

In [ ]:
#将回答和问题信息按照qid进行合并
ans_topic_vector = pd.merge(ans_topic,ques_topic, on='qid')
del ques

#ans对于文本信息只留了topic

In [ ]:
ans_topic_vector.head()

In [ ]:
print(ans_topic_vector.shape)

In [ ]:
def parse_str(d):
    return np.array(list(map(float, d.split())))

In [ ]:
with open('pkl/topic.pkl', 'rb') as file:
    topicmap = pickle.load(file)
topicmap.shape

In [ ]:
topic_vector_dict = dict(zip(np.array(topicmap['id']), np.array(topicmap['embed'])))

type(topic_vector_dict.keys())

In [ ]:
#求话题向量平均值
def topic2v(x):
    try:
        tmp = topic_vector_dict[x[0]]
    except:
        tmp = np.zeros(64)
    for i in x[1:]:
        tmp = tmp + topic_vector_dict[i]
    if len(tmp) == 0:
        return np.zeros(64)
    return (tmp / len(x))

In [ ]:
tqdm.pandas(desc="topic2v...")
ans_topic_vector['topic_vector']=ans_topic_vector['topic'].progress_apply(lambda x:topic2v(x))
print('finished!')

In [ ]:
ans_topic_vector.head()

In [ ]:
with open('pkl/user_feat.pkl', 'rb') as file:
    user = pickle.load(file)
user.shape

In [ ]:
columns=['uid']

columns.append('inter_topic_vector')
user_topic=user[columns]
user_topic.head()

In [ ]:
answer_q_topic_vector = pd.merge(user_topic,ans_topic_vector, on='uid')

In [ ]:
answer_q_topic_vector.head()

In [ ]:
for i in range(64):
    col_name = 'answer_q_topic_vector_{}'.format(str(i))
    tqdm.pandas(desc="topic_interest2v...")
    ans_topic_vector[col_name]=ans_topic_vector['topic_vector'].apply(lambda x:listi(x,i))